# simple Gen AI APP Using  Langchain

In [1]:
# setup key environment
import  os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
# langsmith tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="True"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# data ingeston :- from the website we need to scrap the data
from langchain_community.document_loaders import WebBaseLoader


c:\Users\siddh\anaconda3\envs\langenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://www.geeksforgeeks.org/nlp/introduction-to-langsmith/")
loader

### steps
 ###### laod data->docs-> divide our text inot chunks -> text -> vector -> vector embedding -> store in vectorstore DB

In [4]:
#make document
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.geeksforgeeks.org/nlp/introduction-to-langsmith/', 'title': 'Introduction to Langsmith - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntroduction to Langsmith - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPractice

In [5]:
# divide document into chunk
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=text_splitter.split_documents(docs)
document



[Document(metadata={'source': 'https://www.geeksforgeeks.org/nlp/introduction-to-langsmith/', 'title': 'Introduction to Langsmith - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="Introduction to Langsmith - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPracticePractice Coding ProblemsNation Skillup- Free CoursesProblem of the DayBuild AI Agent (Free)StudyIn: Gl

In [6]:
# embedding
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()


In [7]:
# vector database
from langchain_community.vectorstores import FAISS
vectorstoreDB=FAISS.from_documents(documents=document,embedding=embedding)

vectorstoreDB

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [8]:
# write query from a vector db
query="Key Features of Langsmith:"
result=vectorstoreDB.similarity_search(query=query)
result[0]

NameError: name 'vectorstoreDB' is not defined

In [ ]:
# retrival chain,document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain


ModuleNotFoundError: No module named 'langchain.chains'

In [10]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    """ 
        Answer the following question based only on the provided context":
        <context>
        {context}
        </context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt)

ModuleNotFoundError: No module named 'langchain.chains'

In [13]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Key Features of Langsmith:",
    "context":[Document(page_content="Key Features of Langsmith:Model Management: Easily manage multiple versions of your language models, track their performance and update them seamlesslyWorkflow Structure: Design complex NLP workflows using a visual interface or code, enabling the integration of various models and services.Deployment: Deploy models and workflows to different environments with minimal configuration.Scalability: Scale your applications horizontally to handle large data and requests.Monitoring and Logging: Keep track of model performance, usage statistics, and error logs for better observability. ")]
}) # i have add context manually

NameError: name 'document_chain' is not defined

### however we want the document to first come from the retiever we just set up.That way , we can use the retriever to dynamically select the moset relevant document and pass those in far a given question.

In [ ]:
## Retriever
# it is a interface and resplsible to get
#  the data to query from vectorstoreDB even we dont do similarity search
retiever=vectorstoreDB.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retiever_chian=create_retrieval_chain(retiever,document_chain)






NameError: name 'vectorstoreDB' is not defined

In [ ]:
# get the response from the llm
responce=retiever_chian.invoke({"input":"Key Features of Langsmith"})
respnce["answer"]

NameError: name 'retiever_chian' is not defined

In [ ]:
responce